# Overview

I found a fun short project recently, set by the online fashion company Zalando to people applying for their Data Science positions. The goal of the project is to locate the whereabouts of a new analyst that they wish to recruit, given some limited information about her.

In this notebook I’ll go through my implementation, both the python code and the small amount of maths needed. In the course of doing research for the project I found a couple of nice alternative solutions: see the blogs of [Aonghus Lawlor](http://aonghuslawlor.com/blog/zalando-challenge.html) (no relation) and [Yaser Martinez](http://yasermartinez.com/zalando/report.html). I have to admit both of them have produced a much more visually appealing result than my code can manage currently, however we do get similar results for the analyst location. I had a lot of issues with the map tiling needed to generate a result similar to theirs, when I learn more about them I hope to come back to this mini project and fix it.

The problem itself involves modelling three probability distributions over a Cartesian plane that represents Berlin. By combining these distributions and finding the maxima it’s possible to figure out the most likely location(s). The solution below is laid out as follows:
1. For self-containment I’ll first recap the challenge brief as it is set by Zalando.
2. Probability Distributions for the three data sources.
3. Shortest Euclidean distance from a point to a line segment
4. Coordinate Transform. Her coordinates are given in terms of latitude and longitude but they’ve been kind enough to give us a decent coordinate transform that takes us to a Cartesian plane (i.e. flat instead of spherical).
5. Python implementation for the probability

## 1 - Challenge Brief

The Zalando Data Intelligence Team is searching for a new top analyst. We already know of an excellent candidate with top analytical and programming skills. Unfortunately, we don’t know her exact whereabouts but we only have some vague information where she might be. Can you tell us where to best send our recruiters and plot an easy to read map of your solution for them?  This is what we could extract from independent sources:

* The candidate is likely to be close to the river Spree. The probability at any point is given by a Gaussian function of its shortest distance to the river. The function peaks at zero and has 95\% of its total integral within +-2730m.
* A probability distribution centred around the Brandenburg Gate also informs us of the candidate’s location. The distribution’s radial profile is log-normal with a mean of 4700m and a mode of 3877m in every direction.
* A satellite offers further information: with 95% probability she is located within 2400 m distance of the satellite’s path (assuming a normal probability distribution).

## 2 - Probability Distributions

Given the information in the challenge brief, three probability distributions ($P_\text{Spree}$, $P_\text{Sat.}$ and $P_\text{BBG}$) are now defined which will later be combined to yield a total probability distribution ($P$). We assume that all three sources of information are equally reliable so that $P = \frac{P_\text{Spree} + P_\text{Sat.} + P_\text{BBG}}{3}$, however they could be weighted later on if desired. 

### Spree distribution parameters

We need to find the parameters of the Gaussian function so we can plot it - these are the standard deviation and the mean, however we are told the mean is zero. We can figure out the standard deviation from the error function and the knowledge that 95\% of the total integral is within a distance 2.73km using the formula 
$$ F(\mu + n \sigma) - F(\mu - n \sigma) = \text{erf} \frac{n}{\sqrt{2}}  $$
where F is the cumulative distribution function of the normal distribution with mean $\mu = 0$, standard deviation $\sigma$ and $n$ varies depending on the confidence interval required, e.g. for 2 standard deviations $n = 2$. As $F(\mu + n \sigma) - F(\mu - n \sigma) = 0.95$ we rearrange the above to find
$$ n = \sqrt{2} \text{erf}^{-1} (0.95) = 1.96 .$$
We know that
$$ n \sigma = 2.730 $$
so
$$ \sigma = \frac{2.730}{1.95996} = 1.39289. $$
 
### Satellite distribution parameters
Similar to the above, we know $n \sigma = 2.4$km so by the same method
$$  \sigma = \frac{2.4}{\text{erf}^{-1}(0.95)} = 1.22451.$$
 
### Brandenburg Gate distribution parameters
Log-normal distributions are parametrised by $\mu$ and $\sigma$, called the location and scale parameters respectively, which we will calculate here for the Brandenburg gate distribution. We are told the mean of this distribution is $m = 4.700$km and the mode $a=3.877$km in all directions. These quantities can be expressed by the formulae:
$$ m = e^{\mu + \frac{\sigma^2}{2}} $$
$$ a = e^{\mu - \sigma^2}. $$
This forms a system of 2 solved equations with 2 unknown parameters ($\mu$ and $\sigma$) and can be solved relatively easily by hand
$$ \mu = \log(a) +  \sigma^2 $$
which is substituted into
\begin{equation*}
\begin{split}
\sigma^2 &= 2 ( \ln{4.7} - \mu ) \\
 &=  2 ( \ln{4.7} - \ln{3.877} ) - 2 \sigma^2   \\
\therefore \sigma &= \sqrt{\frac{2}{3} \ln{\frac{4.7}{3.877}}} \\
&= 0.3582
\end{split}
\end{equation*}
so $ \mu = 1.4834 $.

## 3 - Shortest Euclidean distance from a point to a line segment

Here we will show how to calculate the nearest Euclidean distance between a point $p$ and a line segment passing through points $p_0$ and $p_1$. Consider the point $p$ shown in figure \ref{fig:2} which is joined to the segment by the vectors $\vec{w}_0$ and $\vec{w}_1$. If 
$$\vec{w}_0 \cdot \vec{v} < 0$$
then it follows the closest point to $p$ on the segment must be $p_0$. Similarly if 
$$\vec{w}_1 \cdot \vec{v} > 0$$
then the closest point to $p$ is $p_1$. If $p$ lies somewhere such that neither of these conditions is fulfilled then it must lie perpendicular to a point lying on the segment itself.By parameterising the line segment such that $$\vec{v}(b) = p_1 + b ( p_0 - p_1)$$ 
where 
$$b = \frac{\vec{w}_0 \cdot \vec{v}}{\vec{v} \cdot \vec{v}}$$
it is the possible to find the point on the line closest to $p$. Note the efficiency of this method - only the dot products $\vec{w}_0 \cdot \vec{v}$ and $\vec{v} \cdot \vec{v}$ are required to find the point on the line segment. When this point is known the Euclidean distance to point $p$ can be calculated easily.

## 4 - Coordinate Transform 

First a coordinate transform function t is defined which can convert between Latitude/Longitude and Cartesian (or vice versa by setting inv=True). This transform was provided to us in the problem brief on the Zalando website and is quite accurate within the Berlin area. 

In [2]:
import numpy as np

def t(xx,yy,inv=False):
    """
    Coordinate transform between Lat/Lon <--> Cartesian
    accurate for the Berlin Area.
    """
    SW_lat = 52.464011 #(Latitude)
    SW_lon = 13.274099 #(Longitude)

    if inv:     # Lat/Lon -> Cartesian
        lon = SW_lon + (xx/111.323)/(np.cos(SW_lat*np.pi/180.0))
        lat = (yy/111.323) + SW_lat
        return lat, lon
    else:       # Cartesian -> Lat/Lon
        x = (yy - SW_lon) * np.cos(SW_lat * np.pi / 180.0) * 111.323
        y = (xx - SW_lat) * 111.323
        return x, y